In [ ]:
!pip install huggingface_hub > /dev/null 2>&1
!pip install transformers > /dev/null 2>&1
!pip install accelerate > /dev/null 2>&1
!pip install sentencepiece > /dev/null 2>&1
!pip install timm > /dev/null 2>&1
!pip install einops > /dev/null 2>&1
!pip install bitsandbytes accelerate > /dev/null 2>&1
!pip install langchain langchain-community sentence-transformers chromadb pypdf unstructured pysqlite3-binary > /dev/null 2>&1
!pip install langchain-experimental > /dev/null 2>&1
!pip install minio > /dev/null 2>&1

In [ ]:
import logging
import sys
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
import gc
import requests
from minio import Minio
from minio.error import S3Error
import nltk
import ssl
__import__('pysqlite3')
sys.modules['sqlite3'] = sys.modules['pysqlite3']
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFLoader # Pour charger des PDF
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.llms import HuggingFacePipeline # Pour envelopper votre modèle HF pour LangChain
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.vectorstores import Chroma
from langchain.retrievers import ParentDocumentRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.retrievers import BaseRetriever
from typing import List
from urllib.parse import urlparse

In [ ]:
logging.getLogger("transformers.generation.utils").setLevel(logging.ERROR)

In [ ]:
nltk_data_dir = "./nltk_data"

if not os.path.exists(nltk_data_dir):
    print(f"Le dossier '{nltk_data_dir}' est introuvable.")
    print("Tentative de téléchargement des paquets NLTK requis ('punkt', 'stopwords')...")
    
    os.makedirs(nltk_data_dir)
    
    try:
        _create_unverified_https_context = ssl._create_unverified_context
    except AttributeError:
        pass
    else:
        ssl._create_default_https_context = _create_unverified_https_context

    nltk.download('punkt', download_dir=nltk_data_dir)
    nltk.download('stopwords', download_dir=nltk_data_dir)
    
    print("Téléchargement des paquets NLTK terminé.")
else:
    print(f"Le dossier '{nltk_data_dir}' existe déjà. Aucune action de téléchargement n'est nécessaire.")

if os.path.abspath(nltk_data_dir) not in nltk.data.path:
    nltk.data.path.append(os.path.abspath(nltk_data_dir))

print("Le script est configuré pour utiliser les paquets NLTK en mode déconnecté.")

In [ ]:
MINIO_BUCKET_NAME = "reda-rag"
Access_key = os.getenv("AWS_ACCESS_KEY_ID")
Secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
s3_endpoint = "192.168.0.150:9000"

LOCAL_MODEL_PATH = "Meta-Llama-3.2-3B-Instruct/"
LLM_LOCAL_PATH = "./models/Meta-Llama-3.2-3B-Instruct"

EMBEDDING_MINIO_PREFIX = "all-MiniLM-L6-v2/"
LOCAL_MODEL_PATH_2 = "./models/all-MiniLM-L6-v2"

def download_model_via_streaming(client, bucket, prefix, local_path):
    """
    Télécharge les fichiers via streaming pour une utilisation mémoire minimale.
    """
    if not os.path.exists(local_path):
        print(f"-> Le dossier '{local_path}' n'existe pas. Début du téléchargement de '{prefix}'...")
        os.makedirs(local_path, exist_ok=True)
        try:
            objects = client.list_objects(bucket, prefix=prefix, recursive=True)
            files_to_download = [obj for obj in objects if not obj.object_name.endswith('/')]

            if not files_to_download:
                print(f"   AVERTISSEMENT : Aucun fichier trouvé sur MinIO avec le préfixe '{prefix}'.")
                return

            for obj in files_to_download:
                file_name = os.path.relpath(obj.object_name, prefix)
                if file_name == 'consolidated.safetensors':
                    file_name = 'model.safetensors'
                
                local_file_path = os.path.join(local_path, file_name)
                
                if not os.path.exists(os.path.dirname(local_file_path)):
                    os.makedirs(os.path.dirname(local_file_path))
                
                print(f"   Téléchargement en streaming de '{obj.object_name}'...")
                
                response = None
                try:
                    response = client.get_object(bucket, obj.object_name)
                    with open(local_file_path, 'wb') as file_data:
                        for chunk in response.stream(amt=1024*1024):
                            file_data.write(chunk)
                finally:
                    if response:
                        response.close()
                        response.release_conn()
                        
            print(f"   Téléchargement pour '{local_path}' terminé.")
        except S3Error as exc:
            print(f"   Une erreur S3 est survenue pour {prefix}: {exc}")
            raise
    else:
        print(f"-> Le modèle dans '{local_path}' existe déjà.")

try:
    minio_client = Minio(s3_endpoint, access_key=Access_key, secret_key=Secret_key, secure=False)
    print("Connexion à MinIO réussie.")
    
    download_model_via_streaming(minio_client, MINIO_BUCKET_NAME, LOCAL_MODEL_PATH, LLM_LOCAL_PATH)
    download_model_via_streaming(minio_client, MINIO_BUCKET_NAME, EMBEDDING_MINIO_PREFIX, LOCAL_MODEL_PATH_2)
    
except Exception as e:
    print(f"Une erreur est survenue lors de la phase de téléchargement : {e}")
    sys.exit(1)

print("\n Tous les modèles sont vérifiés et prêts localement.")

In [ ]:
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
print(f"Chargement du tokenizer depuis le chemin local : {LLM_LOCAL_PATH}...")
tokenizer = AutoTokenizer.from_pretrained(
    LLM_LOCAL_PATH,
    trust_remote_code=True,
    local_files_only=True
)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [ ]:
print(f"Chargement du modèle depuis {LLM_LOCAL_PATH} avec quantisation 4-bit...")
model = AutoModelForCausalLM.from_pretrained(
    LLM_LOCAL_PATH,
    quantization_config=nf4_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    local_files_only=True
)
print("Modèle chargé avec succès sur le GPU depuis le volume local !")

In [ ]:
embeddings = SentenceTransformerEmbeddings(model_name=LOCAL_MODEL_PATH_2, model_kwargs={'device': 'cuda'} )

In [ ]:
USE_DOCLING_API = False

DOCLING_API_URL = "http://route-increased-junglefowl-docling.apps.neutron-sno-gpu.neutron-it.fr/v1alpha/convert/file"
PDF_DIRECTORY_ON_MINIO = "documents/"
LOCAL_PDF_DOWNLOAD_DIR = "./pdf_downloads/"

all_documents = []
os.makedirs(LOCAL_PDF_DOWNLOAD_DIR, exist_ok=True)

try:
    print(f"Recherche des fichiers PDF dans le dossier MinIO: '{PDF_DIRECTORY_ON_MINIO}'...")
    pdf_objects = minio_client.list_objects(MINIO_BUCKET_NAME, prefix=PDF_DIRECTORY_ON_MINIO, recursive=True)
    pdf_object_names = [obj.object_name for obj in pdf_objects if obj.object_name.lower().endswith('.pdf')]
    print(f"{len(pdf_object_names)} document(s) PDF trouvé(s) pour traitement.")

    if not pdf_object_names:
        raise Exception("Aucun fichier PDF n'a été trouvé. Vérifiez le chemin sur MinIO.")
    print("-" * 40)
    for pdf_object_name in pdf_object_names:
        #print("-" * 40)
        #print(f"Traitement du fichier : '{pdf_object_name}'")
        
        local_pdf_path = os.path.join(LOCAL_PDF_DOWNLOAD_DIR, os.path.basename(pdf_object_name))
        if not os.path.exists(local_pdf_path):
            print(f"  -> Téléchargement vers '{local_pdf_path}'...")
            minio_client.fget_object(MINIO_BUCKET_NAME, pdf_object_name, local_pdf_path)

        if USE_DOCLING_API:
            print("  -> Méthode choisie : API Docling.")
            with open(local_pdf_path, 'rb') as f:
                pdf_content = f.read()
            
            files_payload = {'files': (os.path.basename(local_pdf_path), pdf_content, 'application/pdf')}
            data_payload = {'output_format': 'text'}
            
            response = requests.post(DOCLING_API_URL, files=files_payload, data=data_payload)
            response.raise_for_status()
            api_data = response.json()

            if 'document' in api_data and 'md_content' in api_data['document'] and api_data['document']['md_content']:
                extracted_text = api_data['document']['md_content']
                all_documents.append(Document(page_content=extracted_text, metadata={"source": pdf_object_name}))
                print(f"  -> Texte extrait avec succès pour '{pdf_object_name}'.")
            else:
                print(f"  -> ERREUR : La réponse de l'API pour '{pdf_object_name}' est invalide. Fichier ignoré.")
        else:
            #print("  -> Méthode choisie : Traitement local avec PyPDFLoader.")
            loader = PyPDFLoader(local_pdf_path)
            documents_from_this_pdf = loader.load()
            all_documents.extend(documents_from_this_pdf)
            #print(f"  -> Document chargé localement. {len(documents_from_this_pdf)} pages ajoutées.")
            
except Exception as e:
    error_details = e.response.text if hasattr(e, 'response') else str(e)
    raise Exception(f"Une erreur est survenue lors du traitement des PDF : {error_details}")

if not all_documents:
    raise Exception("Aucun document n'a pu être traité. Arrêt du script.")
    
print(f"Traitement terminé. Nombre total de documents/pages prêt(s) pour le RAG : {len(all_documents)}")

In [ ]:
print("Découpage des documents en chunks sémantiques...")

text_splitter = SemanticChunker(embeddings, breakpoint_threshold_type="percentile")
all_chunks = text_splitter.split_documents(all_documents)

print(f"Nombre total de chunks créés : {len(all_chunks)}")

In [ ]:
print("Initialisation du Vector Store (ChromaDB) avec les chunks...")

vectorstore = Chroma.from_documents(documents=all_chunks, embedding=embeddings)

print("Vector Store prêt !")

In [ ]:
class NeighborRetriever(BaseRetriever):
    """
    Un retriever qui trouve le meilleur document et retourne ce document
    plus son voisin d'avant et son voisin d'après.
    """
    vectorstore: Chroma
    all_docs: List[Document]

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        
        # A. On trouve le meilleur chunk correspondant à la question
        best_docs = self.vectorstore.similarity_search(query, k=1)
        if not best_docs:
            return []

        best_doc_content = best_docs[0].page_content
        
        # B. On retrouve son index dans la liste complète des chunks
        try:
            best_doc_index = [doc.page_content for doc in self.all_docs].index(best_doc_content)
        except ValueError:
            # Si on ne le trouve pas (très rare), on retourne juste le meilleur doc
            return best_docs

        # C. On définit la "fenêtre" : 1 avant, 1 après
        start_index = max(0, best_doc_index - 1)
        end_index = min(len(self.all_docs), best_doc_index + 2) # +2 car la tranche est exclusive

        # D. On retourne les 3 chunks (ou 2 si c'est un bord)
        return self.all_docs[start_index:end_index]

# --- 4. On crée une instance de notre nouveau retriever ---
print("Initialisation du retriever personnalisé 'Voisins'...")
retriever = NeighborRetriever(
    vectorstore=vectorstore,
    all_docs=all_chunks
)
print("Retriever à contexte enrichi (voisins) prêt !")

In [ ]:
text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=4096,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    return_full_text=False
)
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [ ]:
#prompt = ChatPromptTemplate.from_template("""[INST] Tu es un assistant qui répond uniquement en te basant sur le CONTEXTE FOURNI.
#Lis attentivement le contexte avant de répondre.
#Si la réponse à la question de l'utilisateur NE SE TROUVE PAS dans le contexte, réponds UNIQUEMENT 'Je ne peux pas répondre à cette question avec le contexte donné.'
#NE RÉPONDS JAMAIS sur la base de tes connaissances préalables.

#Contexte:
#{context}

#Question: {input} [/INST]""")

prompt_template_str = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Tu es un assistant utile et bienveillant. Ton objectif est de fournir des réponses complètes et précises.

Pour répondre à la question de l'utilisateur, réfère-toi d'abord au CONTEXTE FOURNI.
Si la réponse NE SE TROUVE PAS dans le contexte, utilise alors tes propres connaissances pour y répondre.
Si tu ne connais pas la réponse, qu'elle soit dans le contexte ou non, dis simplement que tu ne sais pas.<|eot_id|><|start_header_id|>user<|end_header_id|>
Contexte:
{context}

Question: {input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
prompt = ChatPromptTemplate.from_template(prompt_template_str)
def format_llama_direct_prompt(question):
    messages = [
        {"role": "user", "content": question}
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [ ]:
document_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
#retriever = vector_store.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
def transform_query_with_llm(question: str, model, tokenizer) -> str:
    """
    Utilise le LLM pour générer plusieurs reformulations fidèles de la question de l'utilisateur.
    """
    transformation_prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    Tu es un outil de reformulation de requêtes. Ton seul rôle est de réécrire la question de l'utilisateur pour la rendre optimale pour une recherche sémantique.
    Règles strictes:
    1. Ne réponds JAMAIS à la question.
    2. Garde le sens original de la question.
    3. Ta sortie doit être une et une seule question.
    4. La question reformulée doit être concise.
    
    Exemple 1:
    Utilisateur: infos sur la sécurité openshift
    Assistant: Quelles sont les meilleures pratiques de sécurité pour un cluster OpenShift ?
    
    Exemple 2:
    Utilisateur: tu peux me faire un résumé ?
    Assistant: Quel est le résumé du document fourni ?
    
    Exemple 3:
    Utilisateur: c'est quoi les grands titre dont tu peux m'aider, en se basant sur le contexte?
    Assistant: Quels sont les thèmes principaux abordés dans le document ?<|eot_id|><|start_header_id|>user<|end_header_id|>
    {question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """
    
    formatted_prompt = transformation_prompt_template.format(question=question)
    
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)
    
    outputs = model.generate(
        **inputs, 
        max_new_tokens=150,
        do_sample=True,
        temperature=0.2,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id
    )
    
    input_length = inputs.input_ids.shape[1]
    new_tokens = outputs[0][input_length:]
    transformed_question = tokenizer.decode(new_tokens, skip_special_tokens=True)
    
    return transformed_question.strip()

In [ ]:
def format_llama_direct_prompt(question):
    messages = [
        {"role": "user", "content": question}
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)


In [ ]:
print("\n\n Système RAG prêt. Posez une question (tapez 'exit' pour quitter) :")
while True:
    try:
        prompt_text = input(">>> ")
        if prompt_text.lower() == 'exit':
            print("Fermeture.")
            break
            
        print("\n--- Transformation de la question par le LLM ---")
        transformed_query = transform_query_with_llm(prompt_text, model, tokenizer)
        print(f"Question transformée : '{transformed_query}'")
        
        print("\n--- Réponse du Modèle Llama (Connaissances Générales) ---")
        
        direct_prompt_formatted = format_llama_direct_prompt(transformed_query)
        
        encodeds_direct = tokenizer(direct_prompt_formatted, return_tensors="pt")
        model_inputs_direct = encodeds_direct.to(model.device)

        outputs_direct = model.generate(
            input_ids=model_inputs_direct.input_ids,
            attention_mask=model_inputs_direct.attention_mask,
            max_new_tokens=4096,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id
        )
        
        input_length = model_inputs_direct.input_ids.shape[1]
        new_tokens = outputs_direct[0][input_length:]
        answer_direct = tokenizer.decode(new_tokens, skip_special_tokens=True)
        
        print(f"Modèle Llama (Direct) : {answer_direct.strip()}")

        print("\n--- Réponse du RAG (Basée sur le Contexte du Fichier) ---")
        response_rag = retrieval_chain.invoke({"input": transformed_query})
        print(f"Modèle RAG : {response_rag['answer'].strip()}")

    except KeyboardInterrupt:
        print("\nFermeture.")
        break
    except Exception as e:
        print(f"Une erreur est survenue : {e}")
        import traceback
        traceback.print_exc()
        break